In [3]:
import numpy as np

In [4]:
sudoku_matrix = np.array([
    [8, 0, 0, 9, 0, 0, 0, 1, 6],
    [0, 0, 9, 0, 0, 8, 0, 0, 0],
    [0, 0, 0, 0, 0, 4, 9, 0, 3],
    [0, 0, 7, 0, 0, 0, 0, 5, 0],
    [0, 0, 0, 0, 6, 0, 0, 0, 0],
    [0, 0, 0, 0, 3, 2, 1, 0, 8],
    [3, 0, 0, 0, 0, 0, 0 ,0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 5, 7, 9, 0]
                          ])
print(sudoku_matrix)

[[8 0 0 9 0 0 0 1 6]
 [0 0 9 0 0 8 0 0 0]
 [0 0 0 0 0 4 9 0 3]
 [0 0 7 0 0 0 0 5 0]
 [0 0 0 0 6 0 0 0 0]
 [0 0 0 0 3 2 1 0 8]
 [3 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 7 9 0]]


In [5]:
def possible_numbers(sud_mat, i, j):
    
    possible_numb = set([k for k in range(1, 10)])
    
    for number in sud_mat[i, :]:
        possible_numb.discard(number)
    for number in sud_mat[:, j]:
        possible_numb.discard(number)
        
    low_i = i - i % 3
    low_j = j - j % 3
    for k in range(low_i, low_i + 3):
        for l in range(low_j, low_j + 3):
            number = sud_mat[k, l]
            possible_numb.discard(number)
            
    return possible_numb

In [6]:
possible_numbers(sudoku_matrix, 8, 1)

{2, 4, 6, 8}

In [7]:
def solve_sudoku(sud_mat, solutions=None):
    if solutions is None:
        solutions = []
    
    for i in range(9):
        for j in range(9):
            if sud_mat[i, j] == 0:
                pos_num = possible_numbers(sud_mat, i, j)
                for number in pos_num:
                    sud_mat[i, j] = number
                    solutions = solve_sudoku(sud_mat, solutions)
                sud_mat[i, j] = 0
                return solutions
    solutions.append(sud_mat.copy())
    return solutions

In [8]:
sol = solve_sudoku(sudoku_matrix.copy())
print('These are the solutions')
print(sol)
print(len(sol))

These are the solutions
[array([[8, 4, 2, 9, 7, 3, 5, 1, 6],
       [5, 3, 9, 6, 1, 8, 2, 4, 7],
       [7, 6, 1, 2, 5, 4, 9, 8, 3],
       [6, 1, 7, 8, 4, 9, 3, 5, 2],
       [2, 8, 3, 5, 6, 1, 4, 7, 9],
       [9, 5, 4, 7, 3, 2, 1, 6, 8],
       [3, 7, 5, 1, 9, 6, 8, 2, 4],
       [1, 9, 8, 4, 2, 7, 6, 3, 5],
       [4, 2, 6, 3, 8, 5, 7, 9, 1]])]
1


In [ ]:
def solve_sudoku_count_sol(sud_mat, stopping_point=30, count=0):
    for i in range(9):
        for j in range(9):
            if sud_mat[i, j] == 0:
                pos_num = possible_numbers(sud_mat, i, j)
                for number in pos_num:
                    sud_mat[i, j] = number
                    count = solve_sudoku_count_sol(sud_mat, stopping_point=stopping_point, count=count)
                sud_mat[i, j] = 0
                return count
    count += 1
    if count > stopping_point:
        raise RuntimeError('Too many solutions')
    print(f'\tFound {count} solutions so far')
    return count

In [10]:
print(sudoku_matrix)
solve_sudoku_count_sol(sudoku_matrix.copy())

[[8 0 0 9 0 0 0 1 6]
 [0 0 9 0 0 8 0 0 0]
 [0 0 0 0 0 4 9 0 3]
 [0 0 7 0 0 0 0 5 0]
 [0 0 0 0 6 0 0 0 0]
 [0 0 0 0 3 2 1 0 8]
 [3 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 7 9 0]]


1

In [11]:
solve_sudoku_count_sol(np.zeros((9, 9), int))

RuntimeError: Too many solutions

In [23]:
def create_sudoku_and_solution(number_of_prefilled=30, max_n_solutions=1, one_sol=False, seed=None):
    """
    If one_sol is True then the number of prefilled will increase until there is just one solution
    """
    rng = np.random.default_rng(seed=seed)
    num_sol = -1
    n_try  = 0
    while num_sol > max_n_solutions or num_sol < 1:
        sud_mat = np.zeros((9, 9), int)
        for _ in range(number_of_prefilled):
            coordinates = rng.integers(9, size=2)
            possibillities = possible_numbers(sud_mat, coordinates[0], coordinates[1])
            while sud_mat[coordinates[0], coordinates[1]] != 0 or len(possibillities) == 0:
                coordinates = rng.integers(9, size=2)
                possibillities = possible_numbers(sud_mat, coordinates[0], coordinates[1])
            sud_mat[coordinates[0], coordinates[1]] = rng.choice(list(possibillities))
        try:
            print('Trying to solve')
            num_sol = solve_sudoku_count_sol(sud_mat, stopping_point=max_n_solutions)
        except RuntimeError:
            num_sol = -1
        n_try += 1    
        print(f'n_try: {n_try}, num_sol: {num_sol}')
    
    # ---This part is extra
    if one_sol is True:
        while num_sol != 1:
            print('Reseting to find one solution')
            sud_mat_2 = sud_mat.copy()
            while num_sol > 1:
                while sud_mat_2[coordinates[0], coordinates[1]] != 0 or len(possibillities) == 0:
                    coordinates = rng.integers(9, size=2)
                    possibillities = possible_numbers(sud_mat_2, coordinates[0], coordinates[1])
                print(f'\tAdding a number')
                sud_mat[coordinates[0], coordinates[1]] = rng.choice(list(possibillities))
        
        
    # ---This part is extra
        
    sol_sud_mat = solve_sudoku(sud_mat)
    return sud_mat, sol_sud_mat, num_sol

    



In [ ]:
sudok, sol_sudok, num = create_sudoku_and_solution(25, max_n_solutions=20, one_sol=True)
print(sudok)
print(sol_sudok[0])
print(num)

Trying to solve
n_try: 1, num_sol: 0
Trying to solve
Found 1 solutions so far
Found 2 solutions so far
Found 3 solutions so far
Found 4 solutions so far
Found 5 solutions so far
Found 6 solutions so far
Found 7 solutions so far
Found 8 solutions so far
Found 9 solutions so far
Found 10 solutions so far
Found 11 solutions so far
Found 12 solutions so far
Found 13 solutions so far
Found 14 solutions so far
Found 15 solutions so far
Found 16 solutions so far
Found 17 solutions so far
Found 18 solutions so far
Found 19 solutions so far
Found 20 solutions so far
n_try: 2, num_sol: -1
Trying to solve
n_try: 3, num_sol: 0
Trying to solve
n_try: 4, num_sol: 0
Trying to solve
